# mlx-whisper

- option1 tiny
`pip install mlx-whisper`

- option2 
  - tiny
  - tiny_en
' pip install -U openai-whisper '



In [16]:
import mlx_whisper

samples_folder='../samples'
import time
import librosa
import os

# Get all audio files from samples folder
audio_files = [f for f in os.listdir(samples_folder) if f.endswith(('.wav', '.flac', '.mp3'))]
print ('default model: whisper-tiny 74m' )


for audio_file in audio_files:
    print(f"\nProcessing: {audio_file}")
    print("-" * 50)
    
    # Get audio duration
    audio_path = os.path.join(samples_folder, audio_file)
    duration = librosa.get_duration(path=audio_path)

    # Transcribe audio
    start_time = time.time()
    text = mlx_whisper.transcribe(audio_path)['text']
    end_time = time.time()

    # Print results
    print(f"Audio duration: {duration:.2f} seconds")
    print(f"Transcription time: {end_time - start_time:.2f} seconds")
    print(f"Transcribed text: {text}")



default model: whisper-tiny 74m

Processing: Speaker27_000.wav
--------------------------------------------------
Audio duration: 60.00 seconds
Transcription time: 0.67 seconds
Transcribed text:  The story of the invention, development, and present-day uses of wars and newest weapons. This is the Librevox Recording. All Librevox Recording are in the public domain. For more information or to volunteer, please visit Librevox.org. Recording by William Tomko. Aircraft and submarines by Willis J. Abbott. Preface. Not since Gunpowder was first employed in warfare has so revolutionary contribution to the science of slaughtering men been made as by the perfection of aircraft and submarines. The former have had their first employment in this worldwide war of the nations. The latter, though in the experimental stage as far back as the American Revolution, have in this bitter contest then for the first time brought to so practical and stage of development as to exert a really appreciable influenc

In [20]:
import whisper

import time
import librosa
import os
samples_folder='../samples'
model = whisper.load_model("tiny")
# Get all audio files from samples folder
audio_files = [f for f in os.listdir(samples_folder) if f.endswith(('.wav', '.flac', '.mp3'))]

print ('openai-whisper, tiny model, 72M')
for audio_file in audio_files:
    print(f"\nProcessing: {audio_file}")
    print("-" * 50)
    
    # Get audio duration
    audio_path = os.path.join( samples_folder, audio_file)
    duration = librosa.get_duration(path=audio_path)

    # Transcribe audio
    start_time = time.time()
    result = model.transcribe(audio_path)
    end_time = time.time()

    # Print results
    print(f"Audio duration: {duration:.2f} seconds")
    print(f"Transcription time: {end_time - start_time:.2f} seconds")
    print(f"Transcribed text: {result['text']}")



openai-whisper, tiny model, 72M

Processing: Speaker27_000.wav
--------------------------------------------------


/opt/miniconda3/envs/mlx-whisper/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Audio duration: 60.00 seconds
Transcription time: 1.26 seconds
Transcribed text:  The story of the invention, development, and present-day uses of wars and newest weapons. This is the Librevox Recording. All Librevox Recording are in the public domain. For more information or to volunteer, please visit Librevox.org. Recording by William Tomko. Aircraft and submarines by Willis J. Abbott. Preface. Not since Gunpowder was first employed in warfare has so revolutionary contribution to the science of slaughtering men been made as by the perfection of aircraft and submarines. The former have had their first employment in this worldwide war of the nations. The latter, though in the experimental stage as far back as the American Revolution, have in this bitter contest then for the first time brought to so practical and stage of development as to exert a really appreciable influence on the outcome of this.

Processing: Speaker26_000.wav
--------------------------------------------------


/opt/miniconda3/envs/mlx-whisper/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Audio duration: 60.00 seconds
Transcription time: 1.08 seconds
Transcribed text:  Section 0 of esophs fables, a new revised version by esoph. This sleeper box recording is in the public domain. Preface, the following are some of esophs best-loved fables. The Goose with the Golden Eggs. A certain man had the good fortune to possess a Goose that laid him a Golden Egg every day, but dissatisfied with so slow and encommed, and thinking to seize the whole treasure at once, he killed the Goose and cutting her open, found her just what any other Goose would be. Much once more, and loses all. The town mouse and the country mouse. A country mouse invited a town mouse and intimate friend to pay him a visit and portake of his country fair. As they were on the bare-plowed lands, eating their wheat stock.

Processing: ls_test.flac
--------------------------------------------------


/opt/miniconda3/envs/mlx-whisper/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Audio duration: 6.67 seconds
Transcription time: 0.31 seconds
Transcribed text:  Then the good soul openly sorted the boat and she had buoyed so long in secret and bravely stretched on alone.


In [21]:
import whisper

import time
import librosa
import os
samples_folder='../samples'
model_en = whisper.load_model("tiny.en")
# Get all audio files from samples folder
audio_files = [f for f in os.listdir(samples_folder) if f.endswith(('.wav', '.flac', '.mp3'))]

print ('openai-whisper, tiny_en model, 72M')
for audio_file in audio_files:
    print(f"\nProcessing: {audio_file}")
    print("-" * 50)
    
    # Get audio duration
    audio_path = os.path.join( samples_folder, audio_file)
    duration = librosa.get_duration(path=audio_path)

    # Transcribe audio
    start_time = time.time()
    result = model_en.transcribe(audio_path)
    end_time = time.time()

    # Print results
    print(f"Audio duration: {duration:.2f} seconds")
    print(f"Transcription time: {end_time - start_time:.2f} seconds")
    print(f"Transcribed text: {result['text']}")



100%|█████████████████████████████████████| 72.1M/72.1M [00:01<00:00, 69.9MiB/s]


openai-whisper, tiny model, 72M

Processing: Speaker27_000.wav
--------------------------------------------------


/opt/miniconda3/envs/mlx-whisper/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Audio duration: 60.00 seconds
Transcription time: 1.12 seconds
Transcribed text:  Preface of Aircraft and Submarines, the story of the invention, development, and present-day uses of wars and newest weapons. This is the Liber vox recording. All Liber vox recordings are in the public domain. For more information or to volunteer, please visit Libervox.org. Recording by William Tomko, Aircraft and Submarines by Willis J. Abbott, Preface. Not since gunpowder was first employed in warfare has so revolutionary a contribution to the science of slaughtering men been made as by the perfection of aircraft and submarines. The former have had their first employment in this worldwide war of the nations. The latter, though in the experimental stage, as far back as the American Revolution, have in this bitter contest, then for the first time brought to so practical a stage of development, as to exert a really appreciable influence on the outcome of the...

Processing: Speaker26_000.wav
--------------

/opt/miniconda3/envs/mlx-whisper/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Audio duration: 60.00 seconds
Transcription time: 0.97 seconds
Transcribed text:  Section zero of ESOP's fables, a new revised version by ESOP. This sliver box recording is in the public domain. Preface, the following are some of ESOP's best-loved fables. The Goose with the Golden Eggs A certain man had the good fortune to possess a goose that laid him a golden egg every day, but dissatisfied with so slow an income and thinking to seize the whole treasure at once, he killed the Goose and cutting her open, found her just what any other Goose would be. Much once more and loses all. The Town Mouse and the Country Mouse A country mouse invited a town mouse and intimate friend to pay him a visit and partake of his country fair. As they were on the bare plowed lands, eating their wheat stock.

Processing: ls_test.flac
--------------------------------------------------


/opt/miniconda3/envs/mlx-whisper/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Audio duration: 6.67 seconds
Transcription time: 0.23 seconds
Transcribed text:  Then the goods sold openly, shorted the burden she had borne so long in secret and bravely trudged on alone.
